In [ ]:
from extract_features import extract_features
from config import load_config
import pysam
import pandas as pd
from ml_utils import load_tmap_labels
import os

os.chdir("/datadisk1/ixk5174/project_repo/Telos-test/")
print(os.getcwd())

cfg = load_config("project_config/pacbio_ENCFF370NFS_isoquant_config.pkl")
# bam = pysam.AlignmentFile("../" + cfg.bam_file, "rb")  # <-- adjust path if needed
print(cfg.tss_labeled_file)
df_tss = pd.read_csv(cfg.tss_labeled_file, dtype={"chrom": str})
df_tes = pd.read_csv(cfg.tes_labeled_file, dtype={"chrom": str})

df_cov = pd.read_csv(cfg.cov_file, sep="\t")
print(f"df_cov.shape: {df_cov.shape}")
df_label = load_tmap_labels(cfg.tmap_file)
print(f"df_label.shape: {df_label.shape}")

# print()

df = df_cov.merge(df_tss, left_on=["tss_chrom", "tss_pos"], right_on=["chrom", "position"], how="inner")
print(f"After merging df_tss with df_cov, df.shape: {df.shape}")

df = df[["transcript_id", "chrom", "position", "label", "coverage"]]
# df = df.merge(df_tes, left_on=["tes_chrom", "tes_pos"], right_on=["chrom", "position"], how="inner", suffixes=("_tss", "_tes"))
# print(f"After merging df_tes with df, df.shape: {df.shape}")
df = df.merge(df_label, left_on=["transcript_id"], right_on=["transcript_id"], how="inner", suffixes=("_tss", "_tr"))

print(f"After merging df_label with df, df.shape: {df.shape}")

# print(df.head(30))


# comparison between tss and transcript labels
matched_labels = df[df["label_tss"] == df["label_tr"]]
print(f"matched_labels.shape: {matched_labels.shape}")
mismatch_labels = df[df["label_tss"] != df["label_tr"]]
print(f"mismatch_labels.shape: {mismatch_labels.shape}")

# sort by coverage
sorted_df = df.sort_values(by="coverage", ascending=False)
# change print to display all columns without line breaks
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)
print(sorted_df.head(100))
 
# generate precision and recall for different coverage bins like a pr curve
precisions = []
recalls = []
n_total = len(sorted_df)
n_positive = sorted_df['label_tss'].sum()

for k in range(1, n_total + 1, 100):
    # Keep top k highest coverage points
    subset = sorted_df.iloc[:k]
    n_kept = len(subset)
    n_tp = subset['label_tss'].sum()  # true positives in kept subset
    n_fp = n_kept - n_tp  # false positives in kept subset
    
    # Precision = TP / (TP + FP)
    precision = n_tp / n_kept 
    # Recall = TP / (total positive)  
    recall = n_tp / n_positive 
    
    precisions.append(precision)
    recalls.append(recall)

# Create dataframe with PR curve data
pr_df = pd.DataFrame({
    'precision': precisions,
    'recall': recalls
})
print(f"PR curve data shape: {pr_df.shape}")

print(pr_df.sort_values(by="recall", ascending=False).head(30))

# generate precision and recall for different coverage bins like a pr curve for transcript labels
precisions = []
recalls = []
n_total = len(sorted_df)
n_positive = sorted_df['label_tr'].sum()

for k in range(1, n_total + 1, 100):
    subset = sorted_df.iloc[:k]
    n_kept = len(subset)
    n_tp = subset['label_tr'].sum()
    n_fp = n_kept - n_tp
    
    precision = n_tp / n_kept
    recall = n_tp / n_positive
    
    precisions.append(precision)
    recalls.append(recall)
    

pr_df_transcript = pd.DataFrame({
    'precision': precisions,
    'recall': recalls
})

print(pr_df_transcript.head(30))








/datadisk1/ixk5174/project_repo/Telos-test
test_output/pacbio_ENCFF370NFS_isoquant/features/pacbio_ENCFF370NFS_isoquant_tss_labeled.tsv
df_cov.shape: (29293, 14)
df_label.shape: (29293, 2)
After merging df_tss with df_cov, df.shape: (29293, 95)
After merging df_label with df, df.shape: (29293, 6)


KeyError: 'label_transcript'

In [6]:
from ml_utils import load_tmap_labels
from config import load_config
import pandas as pd
import os 
import numpy as np
from ml_utils import stratified_split

os.chdir("/datadisk1/ixk5174/project_repo/Telos-test/")
print(os.getcwd())

project_config = load_config("project_config/cDNA-NA12878_stringtie_config.pkl")
pred_df = pd.read_csv(f"{project_config.predictions_output_dir}/randomforest_stage2_predictions_train.csv", sep = "\t")

# find accuracy of model
acc = 0
inc_count = 0
# pred_df['pred_label'] = pred_df['pred_label'].astype(int)
for i in range(len(pred_df)):
    if pred_df.iloc[i]['pred_label'] == pred_df.iloc[i]['label']:
        acc += 1
    else:
        # print(f"pred_df.iloc[i]:  {pred_df.iloc[i]}")
        inc_count += 1
print(f"Accuracy of model: {acc/len(pred_df)}")
print(f"Inc_count: {inc_count}")




/datadisk1/ixk5174/project_repo/Telos-test
Accuracy of model: 0.8783683821198351
Inc_count: 1151
